census resources:
- example blockgroup: https://api.census.gov/data/2019/pdb/blockgroup/examples.html
- api document: https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
- handbook: https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf
- variables: https://api.census.gov/data/2019/acs/acs5/variables.html
- data surveys: https://api.census.gov/data.html
- query exmaples: https://www.census.gov/data/developers/guidance/api-user-guide/query-examples.html
- geoindentifiers: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
- tutorial: https://github.com/datamade/census
- api intro: https://www.census.gov/data/developers/data-sets/acs-5year.html

In [3]:
import tarfile
import pandas as pd
import numpy as np
import os
import requests
import dotenv

In [4]:
# load key for API
dotenv.load_dotenv()
key = os.getenv('census_key')

In [5]:
# load ems data and get coordinates
emsdata = pd.read_csv('locationData.csv')
emsdata['Coordinates']=list(zip(emsdata['IncidentLatitude'],emsdata['IncidentLongitude']))

### Get block codes from geocoordinates

In [10]:
# API for getting blocks from geocode
# **Be careful running this, it takes about an hour**

# Specify call and parameters
search_url = 'https://geo.fcc.gov/api/census/area?'

# get coordinates for lookup
coord = emsdata[~((emsdata['IncidentLatitude'].isna())|emsdata['IncidentLongitude'].isna())]['Coordinates'].drop_duplicates()

# fetch data and save as json
blockmap = [requests.get(search_url,params={'lat': la, 'lon': lo}) \
     .json()['results'][0]['block_fips'] \
     for la, lo in coord]

In [12]:
# store block level data in dataframe
blocks = pd.DataFrame({
    'Coordinates':coord,
    'BlockGroup':blockmap
})

In [14]:
# merge block data to EMS
emsdata_bg = emsdata.merge(blocks, left_on='Coordinates', right_on='Coordinates')
emsdata_bg['BlockGroup12'] = emsdata_bg['BlockGroup'].str[:12]
emsdata_bg.to_csv('emsmerged.csv')

In [6]:
# export ems data merged with block data
emsdata_merged = pd.read_csv('emsmerged.csv')

### Get census data for blockcodes

In [15]:
# get unique blocks
emsdata_merged['BlockGroup12'] = emsdata_merged['BlockGroup'].astype(str).str[:12]
blockUnique = emsdata_merged['BlockGroup12'].drop_duplicates()

In [32]:
# API for getting census-level data
from census import Census

c = Census(key)

#block group level income data
blockIncome = [c.acs5.get(('NAME', 'B19013_001E'),
          {'for': 'block group:{}'.format(b[11:]),
          'in':'state:{} county:{} tract:{}'.format(b[0:2], 
                                                    b[2:5], 
                                                    b[5:11])})[0]['B19013_001E']
for b in blockUnique]

In [34]:
censusBlock = pd.DataFrame({
    'Block': blockUnique,
    'MedIncome': blockIncome,
})

censusBlock

,Block,MedIncome
0,515400005023,98967.0
1,515400004012,61932.0
2,515400002022,12000.0
3,515400008004,51250.0
4,515400004023,36670.0
...,...,...
20698,510030111003,-666666666.0
21353,510030102012,88672.0
21369,510030105001,-666666666.0
21868,510030111004,99333.0


In [36]:
# merge block data to EMS
emsdata_merged2 = emsdata_merged.merge(censusBlock, left_on='BlockGroup12', right_on='Block')
emsdata_merged2.to_csv('emsMergedIncome.csv')

,Unnamed: 0,Unnamed: 0.1,Number,IncidentAddress,Street,Town,State,Zip,IncidentLatitude,IncidentLongitude,Coordinates,BlockGroup,BlockGroup12,Block,MedIncome
0,0,0,2016-00006845,2627 Jefferson Park Charlottesville VA 22903,2627 Jefferson Park,Charlottesville,VA,22903,38.017795,-78.517822,"(38.017795, -78.517822)",515400005023012,515400005023,515400005023,98967.0
1,1544,1391,2017-00000647,220 STRIBLING C Charlottesville VA 22903,220 STRIBLING C,Charlottesville,VA,22903,38.022427,-78.520697,"(38.022427, -78.520697)",515400005023003,515400005023,515400005023,98967.0
2,2729,2952,2017-00001409,128 A STRIBLING Charlottesville VA 22903,128 A STRIBLING,Charlottesville,VA,22903,38.022747,-78.516936,"(38.022747, -78.516936)",515400005023005,515400005023,515400005023,98967.0
3,3608,4197,2017-00002012,218 STRIBLING 1 Charlottesville VA 22903,218 STRIBLING 1,Charlottesville,VA,22903,38.022439,-78.520650,"(38.022439, -78.52065)",515400005023003,515400005023,515400005023,98967.0
4,3609,6065,2017-00002957,218 Stribling Charlottesville VA 22903,218 Stribling,Charlottesville,VA,22903,38.022439,-78.520650,"(38.022439, -78.52065)",515400005023003,515400005023,515400005023,98967.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24288,21353,68705,2019-00009683,201 BOWEN Charlottesville VA 22911,201 BOWEN,Charlottesville,VA,22911,38.141311,-78.447964,"(38.141311, -78.447964)",510030102012031,510030102012,510030102012,88672.0
24289,21369,68843,2019-00009753,108 WENDOVER Charlottesville VA 22911,108 WENDOVER,Charlottesville,VA,22911,38.061573,-78.442811,"(38.061573, -78.442811)",510030105001015,510030105001,510030105001,-666666666.0
24290,21868,76278,2020-00001098,1445 PEACH TREE Crozet VA 22932,1445 PEACH TREE,Crozet,VA,22932,38.075364,-78.714261,"(38.075364, -78.714261)",510030111004010,510030111004,510030111004,99333.0
24291,22399,85243,2020-00005317,644 BAYWICK Crozet VA 22932,644 BAYWICK,Crozet,VA,22932,38.055967,-78.715474,"(38.055967, -78.715474)",510030111004055,510030111004,510030111004,99333.0
